In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git


  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-0s06aji1
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-0s06aji1
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit 0d2ab99cccbbc682722e708515fe9c4cfc50185a
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4716 sha256=e2a73928d23934aebada905a7345abd21834044ddf4ce3186c77b2a559fb5d69
  Stored in directory: /tmp/pip-ephem-wheel-cache-w9pk2v__/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [10]:
%%cu
#include <stdio.h>

__global__ void vecAdd (float* a, float* b, float* c, int n) {

    int i = threadIdx.x + blockDim.x * blockIdx.x;

    if (i < n) {
        c[i] = a[i] + b[i];
    }
}


%%cu
int main() {
    int n = 5;

    int size = n * sizeof(float);

    float* a_h = {1, 2, 3, 4, 5};
    float* b_h = {1, 2, 3, 4, 5};
    float* c_h = (float*) malloc(size);

    float *a_d;
    float *b_d;
    float *c_d;

    cudaMalloc((void**) &a_d, size);
    cudaMalloc((void**) &b_d, size);
    cudaMalloc((void**) &c_d, size);

    cudaMemcpy(a_d, a_h, size, cudaMemcpyHostToDevice);
    cudaMemcpy(b_d, b_h, size, cudaMemcpyHostToDevice);
    cudaMemcpy(c_d, c_h, size, cudaMemcpyHostToDevice);

    vecAdd <<< ceil(n / 256.0), 256 >> (a_d, b_d, c_d, n);

    cudaMemcpy(c_h, c_d, size, cudaMemcpyDeviceToHost);

    for(int i=0; i<n; ++i) {
        printf("%f ", c_h[i]);
    }

    cudaFree(a_d);
    cudaFree(b_d);
    cudaFree(c_d);

    return 0;
}

UsageError: Cell magic `%%cu` not found.
